# MCP Server Template & Best Practices

**Building Professional MCP Servers from Scratch**

---

Welcome to the **MCP server template** guide! This notebook provides a reusable template and best practices for building production-ready MCP servers. By the end of this 10-minute tutorial, you'll have a solid foundation for creating your own MCP servers.

### 🎯 What You'll Learn

In this tutorial, you will:
- Use the MCP server template effectively
- Follow best practices for tool design
- Implement proper error handling and validation
- Test your MCP servers thoroughly
- Deploy servers to production
- Create a complete MCP server from scratch

### 🏗️ Why Templates Matter

A good template provides:
- **Consistency**: Standard structure across servers
- **Best Practices**: Built-in patterns that work
- **Speed**: Faster development with less boilerplate
- **Quality**: Fewer bugs through proven patterns

## 📦 Step 1: Understanding the Template Structure

### Template Components
Our MCP server template includes these key sections:

1. **Imports and Setup**: Standard dependencies
2. **Server Creation**: FastMCP initialization
3. **Tool Definitions**: Your custom tools
4. **Initialization**: Resource setup
5. **Cleanup**: Graceful shutdown
6. **Main Entry**: Server startup

In [ ]:
# Let's examine the template structure
print("📋 MCP SERVER TEMPLATE STRUCTURE")
print("=" * 50)

template_structure = {
    "1. Header": [
        "Module docstring",
        "Import statements",
        "Type definitions"
    ],
    "2. Configuration": [
        "Environment variables",
        "Default settings",
        "Server metadata"
    ],
    "3. Tools": [
        "Tool decorators",
        "Input validation",
        "Error handling"
    ],
    "4. Lifecycle": [
        "Initialization",
        "Resource management",
        "Cleanup routines"
    ],
    "5. Entry Point": [
        "Main function",
        "Transport setup",
        "Error recovery"
    ]
}

for section, components in template_structure.items():
    print(f"\n{section}:")
    for component in components:
        print(f"   • {component}")

## 🔧 Step 2: Complete MCP Server Template

### The Universal Template
Here's a complete, production-ready MCP server template you can use as a starting point:

In [ ]:
# Display the complete MCP server template
template_code = '''
#!/usr/bin/env python3
"""
MCP Server Template
==================

A production-ready template for creating MCP servers.
Replace this with your server description.

Author: Your Name
Date: 2024
"""

import os
import sys
import logging
import asyncio
from typing import Dict, List, Any, Optional
from datetime import datetime

from mcp.server import FastMCP
from mcp.server.stdio import stdio_server

# ===== CONFIGURATION =====
# Server metadata
SERVER_NAME = "Template Server"
SERVER_VERSION = "1.0.0"

# Environment variables
LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO")
DATA_DIR = os.getenv("DATA_DIR", "./data")

# ===== LOGGING SETUP =====
logging.basicConfig(
    level=getattr(logging, LOG_LEVEL),
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s"
)
logger = logging.getLogger(__name__)

# ===== CREATE MCP SERVER =====
mcp = FastMCP(SERVER_NAME)

# ===== GLOBAL STATE =====
# Initialize any global state your server needs
server_state: Dict[str, Any] = {
    "initialized": False,
    "start_time": datetime.now(),
    "request_count": 0
}

# ===== INITIALIZATION =====
@mcp.startup
async def startup():
    """Initialize server resources."""
    logger.info(f"Starting {SERVER_NAME} v{SERVER_VERSION}")
    
    # Create data directory if needed
    os.makedirs(DATA_DIR, exist_ok=True)
    
    # Initialize your resources here
    # e.g., database connections, API clients, etc.
    
    server_state["initialized"] = True
    logger.info("Server initialized successfully")

# ===== CLEANUP =====
@mcp.shutdown
async def shutdown():
    """Clean up server resources."""
    logger.info("Shutting down server...")
    
    # Clean up your resources here
    # e.g., close database connections, save state, etc.
    
    logger.info("Server shutdown complete")

# ===== TOOL DEFINITIONS =====

@mcp.tool(description="Get server status and statistics")
def get_status() -> Dict[str, Any]:
    """Return current server status."""
    uptime = datetime.now() - server_state["start_time"]
    
    return {
        "server": SERVER_NAME,
        "version": SERVER_VERSION,
        "status": "running" if server_state["initialized"] else "initializing",
        "uptime_seconds": int(uptime.total_seconds()),
        "request_count": server_state["request_count"]
    }

@mcp.tool(description="Example tool - replace with your own")
def example_tool(
    param1: str,
    param2: int = 10,
    param3: Optional[bool] = None
) -> Dict[str, Any]:
    """Example tool implementation.
    
    Args:
        param1: Required string parameter
        param2: Optional integer with default
        param3: Optional boolean parameter
    
    Returns:
        Dict with results
    """
    # Increment request counter
    server_state["request_count"] += 1
    
    # Input validation
    if not param1.strip():
        raise ValueError("param1 cannot be empty")
    
    if param2 < 0 or param2 > 100:
        raise ValueError("param2 must be between 0 and 100")
    
    # Your tool logic here
    result = {
        "input": {
            "param1": param1,
            "param2": param2,
            "param3": param3
        },
        "processed": True,
        "timestamp": datetime.now().isoformat()
    }
    
    logger.info(f"Processed example_tool request: {param1}")
    return result

# ===== MAIN ENTRY POINT =====
def main():
    """Run the MCP server."""
    try:
        # Run the server using stdio transport
        asyncio.run(
            stdio_server(
                mcp,
                log_level=LOG_LEVEL.lower()
            )
        )
    except KeyboardInterrupt:
        logger.info("Server stopped by user")
    except Exception as e:
        logger.error(f"Server error: {e}", exc_info=True)
        sys.exit(1)

if __name__ == "__main__":
    main()
'''

print("📄 COMPLETE MCP SERVER TEMPLATE")
print("=" * 50)
print(template_code)

## 🎯 Step 3: Building a Real-World Example

### Note Manager Server
Let's build a practical note management server using our template:

In [ ]:
# Note Manager MCP Server
from mcp.server import FastMCP
from typing import Dict, List, Any, Optional
from datetime import datetime
import json
import os

# Create the MCP server
note_mcp = FastMCP("Note Manager")

# Storage directory
NOTES_DIR = "./notes"
os.makedirs(NOTES_DIR, exist_ok=True)

# Note ID counter
note_counter = 0

def get_next_id() -> str:
    """Get next note ID."""
    global note_counter
    note_counter += 1
    return f"note_{note_counter:04d}"

def save_note(note_id: str, note: Dict[str, Any]) -> None:
    """Save note to disk."""
    path = os.path.join(NOTES_DIR, f"{note_id}.json")
    with open(path, 'w') as f:
        json.dump(note, f, indent=2)

def load_note(note_id: str) -> Optional[Dict[str, Any]]:
    """Load note from disk."""
    path = os.path.join(NOTES_DIR, f"{note_id}.json")
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    return None

@note_mcp.tool(description="Create a new note")
def create_note(
    title: str,
    content: str,
    tags: List[str] = None
) -> Dict[str, Any]:
    """Create a new note."""
    # Validation
    if not title.strip():
        raise ValueError("Title cannot be empty")
    
    # Create note
    note_id = get_next_id()
    note = {
        "id": note_id,
        "title": title.strip(),
        "content": content,
        "tags": tags or [],
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat()
    }
    
    # Save to disk
    save_note(note_id, note)
    
    return note

@note_mcp.tool(description="List all notes")
def list_notes(
    tag: Optional[str] = None,
    limit: int = 50
) -> List[Dict[str, Any]]:
    """List all notes with optional filtering."""
    notes = []
    
    # Read all note files
    for filename in os.listdir(NOTES_DIR):
        if filename.endswith('.json'):
            note_id = filename[:-5]  # Remove .json
            note = load_note(note_id)
            if note:
                # Filter by tag if specified
                if tag and tag not in note.get('tags', []):
                    continue
                notes.append(note)
    
    # Sort by creation date (newest first)
    notes.sort(key=lambda n: n['created_at'], reverse=True)
    
    return notes[:limit]

@note_mcp.tool(description="Get a specific note")
def get_note(note_id: str) -> Dict[str, Any]:
    """Get a note by ID."""
    note = load_note(note_id)
    if not note:
        raise ValueError(f"Note not found: {note_id}")
    return note

@note_mcp.tool(description="Update a note")
def update_note(
    note_id: str,
    title: Optional[str] = None,
    content: Optional[str] = None,
    tags: Optional[List[str]] = None
) -> Dict[str, Any]:
    """Update an existing note."""
    note = load_note(note_id)
    if not note:
        raise ValueError(f"Note not found: {note_id}")
    
    # Update fields if provided
    if title is not None:
        note['title'] = title.strip()
    if content is not None:
        note['content'] = content
    if tags is not None:
        note['tags'] = tags
    
    note['updated_at'] = datetime.now().isoformat()
    
    # Save updated note
    save_note(note_id, note)
    
    return note

@note_mcp.tool(description="Delete a note")
def delete_note(note_id: str) -> Dict[str, str]:
    """Delete a note."""
    path = os.path.join(NOTES_DIR, f"{note_id}.json")
    if not os.path.exists(path):
        raise ValueError(f"Note not found: {note_id}")
    
    os.remove(path)
    
    return {
        "status": "success",
        "message": f"Note {note_id} deleted"
    }

print("✅ Note Manager MCP Server created!")
print("   Tools: create_note, list_notes, get_note,")
print("          update_note, delete_note")

## 🧪 Step 4: Testing Your MCP Server

### Comprehensive Testing
Always test your MCP server thoroughly before deployment:

In [ ]:
# Testing the Note Manager
print("🧪 TESTING NOTE MANAGER")
print("=" * 50)

# Test 1: Create notes
print("\n1️⃣ Creating Notes")
note1 = create_note(
    title="MCP Server Best Practices",
    content="Always validate inputs and handle errors gracefully.",
    tags=["mcp", "development"]
)
print(f"✅ Created: {note1['title']} (ID: {note1['id']})")

note2 = create_note(
    title="Testing Guidelines",
    content="Test all edge cases and error conditions.",
    tags=["testing", "qa"]
)
print(f"✅ Created: {note2['title']} (ID: {note2['id']})")

# Test 2: List notes
print("\n2️⃣ Listing Notes")
all_notes = list_notes()
print(f"✅ Total notes: {len(all_notes)}")

# Test 3: Filter by tag
print("\n3️⃣ Filtering by Tag")
mcp_notes = list_notes(tag="mcp")
print(f"✅ Notes with 'mcp' tag: {len(mcp_notes)}")

# Test 4: Update note
print("\n4️⃣ Updating Note")
updated = update_note(
    note1['id'],
    content="Always validate inputs, handle errors, and log everything."
)
print(f"✅ Updated: {updated['title']}")

# Test 5: Error handling
print("\n5️⃣ Testing Error Handling")
try:
    get_note("invalid_id")
except ValueError as e:
    print(f"✅ Error handled correctly: {e}")

# Test 6: Delete note
print("\n6️⃣ Deleting Note")
result = delete_note(note2['id'])
print(f"✅ {result['message']}")

## ✅ Step 5: Best Practices Checklist

### Essential Guidelines
Follow these best practices for production-ready MCP servers:

In [ ]:
# Best practices checklist
print("✅ MCP SERVER BEST PRACTICES")
print("=" * 50)

best_practices = {
    "🏗️ Structure": [
        "Use the standard template structure",
        "Organize tools logically",
        "Keep related functions together",
        "Use clear, descriptive names"
    ],
    "🛡️ Input Validation": [
        "Validate all required parameters",
        "Check data types and ranges",
        "Sanitize string inputs",
        "Provide clear error messages"
    ],
    "⚠️ Error Handling": [
        "Use try-except blocks appropriately",
        "Raise meaningful exceptions",
        "Log errors with context",
        "Never expose sensitive information"
    ],
    "📊 Logging": [
        "Log important operations",
        "Use appropriate log levels",
        "Include request context",
        "Avoid logging sensitive data"
    ],
    "🔒 Security": [
        "Never trust user input",
        "Implement rate limiting",
        "Use environment variables for secrets",
        "Follow principle of least privilege"
    ],
    "📈 Performance": [
        "Use async operations when appropriate",
        "Implement caching for expensive operations",
        "Limit response sizes",
        "Monitor resource usage"
    ],
    "📚 Documentation": [
        "Document all tools clearly",
        "Include parameter descriptions",
        "Provide usage examples",
        "Keep README up to date"
    ]
}

for category, practices in best_practices.items():
    print(f"\n{category}:")
    for practice in practices:
        print(f"   • {practice}")

## 🚀 Step 6: Deployment Configuration

### Production Setup
Here's how to deploy your MCP server:

In [ ]:
# Deployment configuration examples
print("🚀 DEPLOYMENT CONFIGURATIONS")
print("=" * 50)

# 1. Package.json for npm deployment
print("\n1️⃣ package.json:")
package_json = '''{
  "name": "mcp-note-manager",
  "version": "1.0.0",
  "description": "MCP server for note management",
  "main": "server.py",
  "scripts": {
    "start": "python server.py",
    "test": "pytest tests/"
  },
  "mcp": {
    "type": "stdio",
    "command": "python",
    "args": ["server.py"]
  }
}'''
print(package_json)

# 2. Requirements.txt
print("\n2️⃣ requirements.txt:")
requirements = '''mcp>=1.0.0
pydantic>=2.0.0
python-dotenv>=1.0.0
pytest>=7.0.0
pytest-asyncio>=0.21.0'''
print(requirements)

# 3. Environment configuration
print("\n3️⃣ .env.example:")
env_example = '''# Server Configuration
LOG_LEVEL=INFO
DATA_DIR=./data

# API Keys (if needed)
API_KEY=your-api-key-here

# Database (if needed)
DATABASE_URL=sqlite:///notes.db'''
print(env_example)

# 4. Docker configuration
print("\n4️⃣ Dockerfile:")
dockerfile = '''FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

CMD ["python", "server.py"]'''
print(dockerfile)

## 🎯 Step 7: Summary & Next Steps

### What We've Learned
You now have everything you need to build professional MCP servers!

In [ ]:
# Summary and next steps
print("🎯 SUMMARY & NEXT STEPS")
print("=" * 50)

print("\n✅ What You've Learned:")
learnings = [
    "MCP server template structure",
    "Tool creation and validation",
    "Error handling and logging",
    "Testing best practices",
    "Deployment configurations",
    "Security considerations"
]
for learning in learnings:
    print(f"   • {learning}")

print("\n🚀 Next Steps:")
next_steps = [
    "1. Copy the template and customize for your needs",
    "2. Add your specific tools and logic",
    "3. Write comprehensive tests",
    "4. Set up proper logging and monitoring",
    "5. Deploy to your environment",
    "6. Monitor and iterate"
]
for step in next_steps:
    print(f"   {step}")

print("\n💡 Pro Tips:")
tips = [
    "Start simple and iterate",
    "Test early and often",
    "Document as you go",
    "Share with the community"
]
for tip in tips:
    print(f"   • {tip}")

print("\n🎉 Congratulations!")
print("You're ready to build production-ready MCP servers!")